# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our objective function. 
After training, we will need these parameters 
in order to make future predictions.
Additionally, we will sometimes wish 
to extract the parameters 
either to reuse them in some other context,
to save our model to disk so that 
it may be exectuted in other software,
or for examination in the hopes of 
gaining scientific understanding.

Most of the time, we will be able 
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on DJL
to do the heavy lifting.
However, when we move away from 
stacked architectures with standard layers, 
we will sometimes need to get into the weeds
of declaring and manipulating parameters. 
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualiziations.
* Parameter initialization.
* Sharing parameters across different model components.

We start by focusing on an MLP with one hidden layer.

In [1]:
%use @file[../djl.json]
// %load ../utils/djl-imports

In [4]:
val manager = NDManager.newBaseManager();

val x = manager.randomUniform(0f, 1f, Shape(2, 4));

val net = SequentialBlock()
      .add(Linear.builder().setUnits(8).build())
      .add(Activation.reluBlock())
      .add(Linear.builder().setUnits(1).build())
      .also {
         it.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT);
         it.initialize(manager, DataType.FLOAT32, x.getShape());
      }

val ps = ParameterStore(manager, false);
net.forward(ps, NDList(x), false).head(); // forward computation

ND: (2, 1) cpu() float32
[[-8.41880683e-05],
 [-1.42212462e-04],
]


## Parameter Access

Let us start with how to access parameters
from the models that you already know.
Each layer's parameters are conveniently stored in a `Pair<String, Parameter>` consisting of a unique
`String` that serves as a key for the layer and the `Parameter` itself.
The `ParameterList` is an extension of `PairList` and is returned with a call to the `getParameters()` method on a `Block`. 
We can inspect the parameters of the `net` defined above.
When a model is defined via the `SequentialBlock` class,
we can access any layer's `Pair<String, Parameter>` by calling `get()` on the `ParameterList` and passing in the index
of the parameter we want. Calling `getKey()` and `getValue()` on a `Pair<String, Parameter>` will get the parameter's name and `Parameter` respectively. We can also directly get the `Parameter` we want from the `ParameterList`
by calling `get()` and passing in its unique key(the `String` portion of the `Pair<String, Parameter>`. If we call `valueAt()` and pass in
the index, we will get the `Parameter` directly as well.

In [5]:
val params = net.getParameters();
// Print out all the keys (unique!)
for (pair in params) {
    println(pair.getKey());
}

// Use the unique key to access the Parameter
val dense0Weight = params.get("01Linear_weight").getArray();
val dense0Bias = params.get("01Linear_bias").getArray();

// Use indexing to access the Parameter
val dense1Weight = params.valueAt(2).getArray();
val dense1Bias = params.valueAt(3).getArray();

println(dense0Weight);
println(dense0Bias);

println(dense1Weight);
println(dense1Bias);

01Linear_weight
01Linear_bias
03Linear_weight
03Linear_bias
weight: (8, 4) cpu() float32 hasGradient
[[ 0.0189, -0.0117, -0.0123,  0.0156],
 [-0.0177, -0.0055, -0.0045, -0.0236],
 [ 0.0058,  0.0054, -0.0186,  0.0268],
 [-0.0198,  0.0125, -0.0021, -0.0055],
 [ 0.0024, -0.0068, -0.0004, -0.0014],
 [-0.0049,  0.0038, -0.0002,  0.0041],
 [ 0.0057,  0.0057,  0.0147, -0.0276],
 [ 0.0069,  0.0108,  0.0035, -0.0061],
]

bias: (8) cpu() float32 hasGradient
[0., 0., 0., 0., 0., 0., 0., 0.]

weight: (1, 8) cpu() float32 hasGradient
[[ 0.0107,  0.0183,  0.0012, -0.0115, -0.0097,  0.0005, -0.0078, -0.0251],
]

bias: (1) cpu() float32 hasGradient
[0.]



The output tells us a few important things.
First, each fully-connected layer 
has two parameters, e.g., 
`dense0Weight` and `dense0Bias`,
corresponding to that layer's 
weights and biases, respectively.
The `params` variable is a `ParameterList` which contain the
key-value pairs of the layer name and a parameter of the 
`Parameter` class.
With a `Parameter`, we can get the underlying numerical values as `NDArray`s by calling 
`getArray()` on them!
Both the weights and biases are stored as single precision floats(`FLOAT32`).


### Targeted Parameters

Parameters are complex objects,
containing data, gradients,
and additional information.
That's why we need to request the data explicitly.
Note that the bias vector consists of zeroes
because we have not updated the network
since it was initialized.

Note that unlike the biases, the weights are nonzero. 
This is because unlike biases, 
weights are initialized randomly. 
In addition to `getArray()`, each `Parameter`
also provides a `requireGradient()` method which
returns whether the parameter needs gradients to be computed
(which we set on the `NDArray` with `attachGradient()`).
The gradient has the same shape as the weight. 
To actually access the gradient, we simply call `getGradient()` on the
`NDArray`.
Because we have not invoked backpropagation 
for this network yet, its values are all 0.
We would invoke it by creating a `GradientCollector` instance and
run our calculations inside it.

In [6]:
dense0Weight.getGradient();

ND: (8, 4) cpu() float32
[[0., 0., 0., 0.],
 [0., 0., 0., 0.],
 [0., 0., 0., 0.],
 [0., 0., 0., 0.],
 [0., 0., 0., 0.],
 [0., 0., 0., 0.],
 [0., 0., 0., 0.],
 [0., 0., 0., 0.],
]


### Collecting Parameters from Nested Blocks

Let us see how the parameter naming conventions work 
if we nest multiple blocks inside each other. 
For that we first define a function that produces Blocks 
(a Block factory, so to speak) and then 
combine these inside yet larger Blocks.

In [7]:
fun  block1() : SequentialBlock {
    val net = SequentialBlock();
    net.add(Linear.builder().setUnits(32).build());
    net.add(Activation.reluBlock());
    net.add(Linear.builder().setUnits(16).build());
    net.add(Activation.reluBlock());
    return net;
}

fun block2() : SequentialBlock{
    val net = SequentialBlock();
    for (i in 0 until 4) {
        net.add(block1());
    }
    return net;
}

val rgnet = SequentialBlock();
rgnet.add(block2());
rgnet.add(Linear.builder().setUnits(10).build());
rgnet.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT);
rgnet.initialize(manager, DataType.FLOAT32, x.getShape());

rgnet.forward(ps, NDList(x), false).singletonOrThrow();

ND: (2, 10) cpu() float32
[[-2.13574868e-14, -3.53356396e-14, -3.29092290e-14, -1.15955321e-14, -5.79714601e-15, -4.31167892e-15, -1.55334645e-14,  3.16186472e-15,  2.84417943e-14, -2.24714559e-15],
 [-2.24051192e-14, -3.47734909e-14, -3.40257878e-14, -1.36719080e-14, -4.52792050e-15, -6.42207506e-15, -1.65834906e-14,  7.80485804e-16,  3.48985028e-14, -1.90861129e-15],
]


Now that we have designed the network, 
let us see how it is organized.
We can get the list of named parameters by calling `getParameters()`.
However, we not only want to see the parameters, but also how
our network is structured.
To see our network architecture, we can simply print out the block whose architecture we want to see.

In [8]:
/* Network Architecture for RgNet */
rgnet

Sequential(
	Sequential(
		Sequential(
			Linear(2 -> 32)
			Lambda()
			Linear(2 -> 16)
			Lambda()
		)
		Sequential(
			Linear(2 -> 32)
			Lambda()
			Linear(2 -> 16)
			Lambda()
		)
		Sequential(
			Linear(2 -> 32)
			Lambda()
			Linear(2 -> 16)
			Lambda()
		)
		Sequential(
			Linear(2 -> 32)
			Lambda()
			Linear(2 -> 16)
			Lambda()
		)
	)
	Linear(2 -> 10)
)

In [9]:
/* Parameters for RgNet */
for (param in rgnet.getParameters()) {
    println(param.getValue().getArray());
}

weight: (32, 4) cpu() float32 hasGradient
[ Exceed max print size ]
bias: (32) cpu() float32 hasGradient
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ... 12 more]

weight: (16, 32) cpu() float32 hasGradient
[ Exceed max print size ]
bias: (16) cpu() float32 hasGradient
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

weight: (32, 16) cpu() float32 hasGradient
[ Exceed max print size ]
bias: (32) cpu() float32 hasGradient
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ... 12 more]

weight: (16, 32) cpu() float32 hasGradient
[ Exceed max print size ]
bias: (16) cpu() float32 hasGradient
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

weight: (32, 16) cpu() float32 hasGradient
[ Exceed max print size ]
bias: (32) cpu() float32 hasGradient
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ... 12 more]

weight: (16, 32) cpu() float32 hasGradient
[ Exceed ma

Since the layers are hierarchically nested,
we can also access them by calling their `getChildren()` method
to get a `BlockList`(also an extension of `PairList`) of their inner blocks.
It shares methods with `ParameterList` and as such we can use their
familiar structure to access the blocks. We can call `get(i)` to get the
`Pair<String, Block>` at the index `i` we want, and then finally `getValue()` to get the actual
block. We can do this in one step as shown above with `valueAt(i)`. Then we have to repeat that to get that blocks child and so on.

Here, we access the first major block, 
within it the second subblock, 
and within that the bias of the first layer,
with as follows:

In [10]:
val majorBlock1 = rgnet.getChildren().get(0).getValue();
val subBlock2 = majorBlock1.getChildren().valueAt(1);
val linearLayer1 = subBlock2.getChildren().valueAt(0);
val bias = linearLayer1.getParameters().valueAt(1).getArray();
bias

bias: (32) cpu() float32 hasGradient
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ... 12 more]


## Parameter Initialization

Now that we know how to access the parameters,
let us look at how to initialize them properly.
We discussed the need for initialization in :numref:`sec_numerical_stability`. 
By default, DJL initializes weight matrices
based on your set initializer 
and the bias parameters are all set to $0$.
However, we will often want to initialize our weights
according to various other protocols. 
DJL's `ai.djl.training.initializer` package provides a variety 
of preset initialization methods.
If we want to create a custom initializer,
we need to do some extra work.

### Built-in Initialization

In DJL, when setting the initializer for blocks, the default `setInitializer()` function does not overwrite
any previous set initializers. So if you set an initializer earlier, but decide you want to change your initializer and call `setInitializer()` again, the second `setInitializer()` will NOT overwrite your first one.

Additionally, when you call `setInitializer()` on a block, all internal blocks will also call `setInitializer()` with the same given `initializer`.

This means that we can call `setInitializer()` on the highest level of a block and know that all internal blocks that do not have an initializer already set will be set to that given `initializer`.

This setup has the advantage that we don't have to worry about our `setInitializer()` overriding our previous `initializer`s on internal blocks!

If you want to however, you can explicitly set an initializer for a `Parameter` by calling its `setInitializer()` function directly.

Let us begin by calling on built-in initializers. The code below initializes all parameters 
to a given constant value 1, by using the `ConstantInitializer()` initializer. 

Note that this will not do anything currently since we have already set
our initializer in the previous code block.
We can verify this by checking the weight of a parameter.

In [ ]:
net.setInitializer(new ConstantInitializer(1), Parameter.Type.WEIGHT);
net.initialize(manager, DataType.FLOAT32, x.getShape());
Block linearLayer = net.getChildren().get(0).getValue();
NDArray weight = linearLayer.getParameters().get(0).getValue().getArray();
weight

We can see these initializations however if we create a new network.
Let us write a function to create these network architectures for us
conveniently.

In [11]:
fun getNet() : SequentialBlock {
    val net = SequentialBlock();
    net.add(Linear.builder().setUnits(8).build());
    net.add(Activation.reluBlock());
    net.add(Linear.builder().setUnits(1).build());
    return net;
}

If we run our previous initializer on this new net and check a parameter, we'll
see that everything is initialized properly! (to 7777!)

In [12]:
val net = getNet();
net.setInitializer(ConstantInitializer(7777f), Parameter.Type.WEIGHT);
net.initialize(manager, DataType.FLOAT32, x.getShape());
val linearLayer = net.getChildren().valueAt(0);
val weight = linearLayer.getParameters().valueAt(0).getArray();
weight

weight: (8, 4) cpu() float32 hasGradient
[[7777., 7777., 7777., 7777.],
 [7777., 7777., 7777., 7777.],
 [7777., 7777., 7777., 7777.],
 [7777., 7777., 7777., 7777.],
 [7777., 7777., 7777., 7777.],
 [7777., 7777., 7777., 7777.],
 [7777., 7777., 7777., 7777.],
 [7777., 7777., 7777., 7777.],
]


We can also initialize all parameters 
as Gaussian random variables 
with standard deviation $.01$.

In [13]:
val net = getNet();
net.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT);
net.initialize(manager, DataType.FLOAT32, x.getShape());
val linearLayer = net.getChildren().valueAt(0);
val weight = linearLayer.getParameters().valueAt(0).getArray();
weight

weight: (8, 4) cpu() float32 hasGradient
[[ 0.0221,  0.0116,  0.0077,  0.0048],
 [ 0.0104,  0.003 ,  0.0118,  0.0015],
 [ 0.0189, -0.0117, -0.0123,  0.0156],
 [-0.0177, -0.0055, -0.0045, -0.0236],
 [ 0.0058,  0.0054, -0.0186,  0.0268],
 [-0.0198,  0.0125, -0.0021, -0.0055],
 [ 0.0024, -0.0068, -0.0004, -0.0014],
 [-0.0049,  0.0038, -0.0002,  0.0041],
]


We can also apply different initializers for certain Blocks.
For example, below we initialize the first layer
with the `XavierInitializer` initializer
and initialize the second layer 
to a constant value of 0.

We will do this without the `getNet()` function as it will be easier
to have the reference to each block we want to set.


In [15]:
val net = SequentialBlock();
val linear1 = Linear.builder().setUnits(8).build();
net.add(linear1);
net.add(Activation.reluBlock());
val linear2 = Linear.builder().setUnits(1).build();
net.add(linear2);

linear1.setInitializer(XavierInitializer(), Parameter.Type.WEIGHT);
linear1.initialize(manager, DataType.FLOAT32, x.getShape());

linear2.setInitializer(Initializer.ZEROS, Parameter.Type.WEIGHT);
linear2.initialize(manager, DataType.FLOAT32, x.getShape());

println(linear1.getParameters().valueAt(0).getArray());
println(linear2.getParameters().valueAt(0).getArray());

weight: (8, 4) cpu() float32 hasGradient
[[ 0.0976,  0.1857,  0.4304,  0.6885],
 [ 0.2055,  0.7159,  0.0898,  0.6945],
 [-0.1527,  0.2471,  0.2918, -0.2312],
 [-0.1248, -0.4049,  0.7835, -0.8866],
 [ 0.9273, -0.4547, -0.2331, -0.0447],
 [ 0.5835,  0.6243,  0.0578, -0.04  ],
 [ 0.1361, -0.2144,  0.8512,  0.6722],
 [-0.8579, -0.3252, -0.8257,  0.2963],
]

weight: (1, 4) cpu() float32 hasGradient
[[0., 0., 0., 0.],
]



Finally, we can directly access the `Parameter.setInitializer()` and set their initializers individually.

In [16]:
val net = getNet();
val params = net.getParameters();

params.get("01Linear_weight").setInitializer(NormalInitializer());
params.get("03Linear_weight").setInitializer(Initializer.ONES);

net.initialize(manager, DataType.FLOAT32, Shape(2, 4));

println(params.valueAt(0).getArray());
println(params.valueAt(2).getArray());

weight: (8, 4) cpu() float32 hasGradient
[[ 0.0221,  0.0116,  0.0077,  0.0048],
 [ 0.0104,  0.003 ,  0.0118,  0.0015],
 [ 0.0189, -0.0117, -0.0123,  0.0156],
 [-0.0177, -0.0055, -0.0045, -0.0236],
 [ 0.0058,  0.0054, -0.0186,  0.0268],
 [-0.0198,  0.0125, -0.0021, -0.0055],
 [ 0.0024, -0.0068, -0.0004, -0.0014],
 [-0.0049,  0.0038, -0.0002,  0.0041],
]

weight: (1, 8) cpu() float32 hasGradient
[[1., 1., 1., 1., 1., 1., 1., 1.],
]



### Custom Initialization

Sometimes, the initialization methods we need 
are not standard in DJL. 
In these cases, we can define a class to implement the `Initializer` interface. 
We only have to implement the `initialize()` function,
which takes an `NDManager`, a `Shape`, and the `DataType`. 
We then create the `NDArray` with the aforementioned `Shape` and `DataType`
and initialize it to what we want! You can also design your
initializer to take in some parameters. Simply declare them
as fields in the class and pass them in as inputs to the constructor!
In the example below, we define an initializer
for the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U[5, 10] & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U[-10, -5] & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

In [17]:
class MyInit : Initializer {


    @Override
    override fun initialize(manager:NDManager , shape:Shape , dataType:DataType ) : NDArray {
        println("Init %s".format(shape.toString()))
        // Here we generate data points 
        // from a uniform distribution [-10, 10]
       val data = manager.randomUniform(-10f, 10f, shape, dataType);
        // We keep the data points whose absolute value is >= 5
        // and set the others to 0.
        // This generates the distribution `w` shown above.
        val absGte5 = data.abs().gte(5); // returns boolean NDArray where 
                                             // true indicates abs >= 5 and
                                             // false otherwise
        return data.mul(absGte5); // keeps true indices and sets false indices to 0.
                                  // special operation when multiplying a numerical
                                  // NDArray with a boolean NDArray
    }

}

In [18]:
val net = getNet();
net.setInitializer(MyInit(), Parameter.Type.WEIGHT);
net.initialize(manager, DataType.FLOAT32, x.getShape());
val linearLayer = net.getChildren().valueAt(0);
val weight = linearLayer.getParameters().valueAt(0).getArray();
weight

Init (8, 4)
Init (1, 8)


weight: (8, 4) cpu() float32 hasGradient
[[ 0.    ,  0.    ,  0.    ,  6.8853],
 [ 0.    ,  7.1589,  0.    ,  6.945 ],
 [-0.    ,  0.    ,  0.    , -0.    ],
 [-0.    , -0.    ,  7.8355, -8.8657],
 [ 9.2733, -0.    , -0.    , -0.    ],
 [ 5.8345,  6.2434,  0.    , -0.    ],
 [ 0.    , -0.    ,  8.5119,  6.7216],
 [-8.5793, -0.    , -8.2574,  0.    ],
]


Note that we always have the option 
of setting parameters directly by calling `getValue().getArray()` 
to access the underlying `NDArray`. 
A note for advanced users: 
you cannot directly modify parameters within a `GarbageCollector` scope.
You must modify them outside the `GarbageCollector` scope to avoid confusing 
the automatic differentiation mechanics.

In [19]:
// '__'i() is an inplace operation to modify the original NDArray
val weightLayer = net.getChildren().valueAt(0)
    .getParameters().valueAt(0).getArray();
weightLayer.addi(7);
weightLayer.divi(9);
weightLayer.set(NDIndex(0, 0), 2020); // set the (0, 0) index to 2020
weightLayer;

weight: (8, 4) cpu() float32 hasGradient
[[ 2.02000000e+03,  7.77777791e-01,  7.77777791e-01,  1.54281282e+00],
 [ 7.77777791e-01,  1.57321250e+00,  7.77777791e-01,  1.54944825e+00],
 [ 7.77777791e-01,  7.77777791e-01,  7.77777791e-01,  7.77777791e-01],
 [ 7.77777791e-01,  7.77777791e-01,  1.64838457e+00, -2.07304537e-01],
 [ 1.80813932e+00,  7.77777791e-01,  7.77777791e-01,  7.77777791e-01],
 [ 1.42605567e+00,  1.47148597e+00,  7.77777791e-01,  7.77777791e-01],
 [ 7.77777791e-01,  7.77777791e-01,  1.72354805e+00,  1.52461946e+00],
 [-1.75475433e-01,  7.77777791e-01, -1.39712647e-01,  7.77777791e-01],
]


## Tied Parameters

Often, we want to share parameters across multiple layers.
Later we will see that when learning word embeddings,
it might be sensible to use the same parameters
both for encoding and decoding words. 
We discussed one such case when we introduced :numref:`sec_model_construction`. 
Let us see how to do this a bit more elegantly. 
In the following we allocate a dense layer 
and then use its parameters specifically 
to set those of another layer.

In [20]:
val net = SequentialBlock();

// We need to give the shared layer a name 
// such that we can reference its parameters
val shared = Linear.builder().setUnits(8).build();
val sharedRelu = SequentialBlock();
sharedRelu.add(shared);
sharedRelu.add(Activation.reluBlock());

net.add(Linear.builder().setUnits(8).build());
net.add(Activation.reluBlock());
net.add(sharedRelu)
net.add(sharedRelu)
net.add(Linear.builder().setUnits(10).build());

val x = manager.randomUniform(-10f, 10f, Shape(2, 20), DataType.FLOAT32);

net.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT);
net.initialize(manager, DataType.FLOAT32, x.getShape());

net.forward(ps, NDList(x), false).singletonOrThrow();

ND: (2, 10) cpu() float32
[[-3.92868242e-06,  3.34342758e-06, -7.98338078e-07,  2.95835832e-07,  4.77646699e-06,  1.38241171e-06,  2.90955313e-08, -9.52185246e-07,  1.51947017e-06,  3.47174478e-06],
 [-6.96133338e-06,  8.60772980e-06, -2.39711335e-06, -1.57033855e-06,  7.75049739e-06,  4.43667705e-06,  9.20264029e-07, -2.27942428e-06,  4.81901225e-06,  7.78320646e-06],
]


In [21]:
// Check that the parameters are the same
val shared1 = net.getChildren().valueAt(2)
    .getParameters().valueAt(0).getArray();
val shared2 = net.getChildren().valueAt(3)
    .getParameters().valueAt(0).getArray();
shared1.eq(shared2);

ND: (8, 8) cpu() boolean
[[ true,  true,  true,  true,  true,  true,  true,  true],
 [ true,  true,  true,  true,  true,  true,  true,  true],
 [ true,  true,  true,  true,  true,  true,  true,  true],
 [ true,  true,  true,  true,  true,  true,  true,  true],
 [ true,  true,  true,  true,  true,  true,  true,  true],
 [ true,  true,  true,  true,  true,  true,  true,  true],
 [ true,  true,  true,  true,  true,  true,  true,  true],
 [ true,  true,  true,  true,  true,  true,  true,  true],
]


This example shows that the parameters 
of the second and third layer are tied. 
They are not just equal, they are 
represented by the same exact `NDArray`. 
Thus, if we change one of the parameters,
the other one changes, too. 
You might wonder, 
*when parameters are tied
what happens to the gradients?*
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
in `shared.getGradient()` during backpropagation.

## Summary

* We have several ways to access, initialize, and tie model parameters.
* We can use custom initialization.
* DJL has a sophisticated mechanism for accessing parameters in a unique and hierarchical manner.


## Exercises

1. Use the FancyMLP defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Look at the [DJL documentation](https://javadoc.io/doc/ai.djl/api/latest/ai/djl/training/initializer/Initializer.html) and explore different initializers.
1. Try accessing the model parameters after `net.initialize()` and before `predictor.predict(x)` to observe the shape of the model parameters. What changes? Why?
1. Construct a multilayer perceptron containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?